In [42]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [44]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! *

In [55]:
import pandas as pd
import numpy as np
import sqlite3 as sql


database = 'sqlite_db_pythonsqlite.db'
connection = sql.connect(database)

query_q10 = '''SELECT Facilities.name, 
	SUM(CASE WHEN Bookings.memid <> 0  
    THEN Facilities.membercost * Bookings.slots 
    ELSE Facilities.guestcost * Bookings.slots 
    END) AS total_revenue
FROM Facilities
INNER JOIN Bookings ON Facilities.facid = Bookings.facid
GROUP BY Facilities.name
HAVING total_revenue < 1000
ORDER BY total_revenue;'''

df_q10 = pd.read_sql_query(query_q10, connection)


print(df_q10)

            name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


In [65]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [67]:
query_q11 = '''SELECT m.surname AS member_surname, m.firstname AS member_firstname, r.surname AS recomender_surname, r.firstname AS recomender_firstname
FROM Members AS m
JOIN Members AS r ON m.recommendedby = r.memid
WHERE m.recommendedby != 0
ORDER BY m.surname, m.firstname;'''

df_q11 = pd.read_sql_query(query_q11, connection)


print(df_q11)

       member_surname member_firstname recomender_surname recomender_firstname
0               Bader         Florence           Stibbons               Ponder
1               Baker             Anne           Stibbons               Ponder
2               Baker          Timothy            Farrell               Jemima
3              Boothe              Tim             Rownam                  Tim
4             Butters           Gerald              Smith               Darren
5              Coplin             Joan              Baker              Timothy
6             Crumpet            Erica              Smith                Tracy
7                Dare            Nancy           Joplette               Janice
8             Genting          Matthew            Butters               Gerald
9                Hunt             John            Purview            Millicent
10              Jones            David           Joplette               Janice
11              Jones          Douglas              

In [69]:
#Q12: Find the facilities with their usage by member, but not guests */

In [59]:
query_q12 = '''SELECT f.name AS facility, m.firstname || ' ' || m.surname AS member_name, m.memid AS member_id
FROM Bookings AS b
INNER JOIN Facilities AS f ON b.facid = f.facid
INNER JOIN Members AS m ON b.memid = m.memid 
WHERE b.memid != 0
ORDER BY facility;'''

df_q12 = pd.read_sql_query(query_q12, connection)

In [61]:
print(df_q12)

             facility   member_name  member_id
0     Badminton Court  Darren Smith          1
1     Badminton Court  Darren Smith          1
2     Badminton Court  Darren Smith          1
3     Badminton Court  Darren Smith          1
4     Badminton Court  Darren Smith          1
...               ...           ...        ...
3155   Tennis Court 2    Tim Boothe          8
3156   Tennis Court 2  Charles Owen         10
3157   Tennis Court 2   David Jones         11
3158   Tennis Court 2  Charles Owen         10
3159   Tennis Court 2    Tim Boothe          8

[3160 rows x 3 columns]


In [71]:
# Q13: Find the facilities usage by month, but not guests */

In [63]:
query_q13 = '''SELECT STRFTIME('%m', starttime) AS month, COUNT(memid) AS facility_usage
FROM Bookings
WHERE memid != 0
GROUP BY month;'''

df_q13 = pd.read_sql_query(query_q13, connection)
print(df_q13)

  month  facility_usage
0    07             480
1    08            1168
2    09            1512
